In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)

In [ ]:
# results_dir = Path("results") / "vulpi"
# results_dir = Path("results") / "husky"
results_dir = Path("../results") / "data_concat"

values = np.load(
    results_dir / "results_tsne_full_CNN_mw_1.7.npy", allow_pickle=True
).item()
# values = np.load(results_dir / "results_CNN_hamming_mw_1.7.npy", allow_pickle=True).item()
# values = np.load(results_dir / "results_tsne_full_CNN_mw_1.7.npy", allow_pickle=True).item()
# values = np.load(results_dir / "results_tsne_CNN_mw_1.7.npy", allow_pickle=True).item()
# values = np.load(results_dir / "results_CNN_optim6_mw_1.7.npy", allow_pickle=True).item()
# values = np.load(results_dir / "results_CNN_hamming_mw_1.7.npy", allow_pickle=True).item()
# values = np.load(results_dir / "results_hamming_aug_CNN_mw_1.7.npy", allow_pickle=True).item()
terrains = np.array([t for t in values["terrains"] if t != "MIXED"])
inv_terrains = {t: i for i, t in enumerate(terrains)}
values

In [ ]:
clf_values = values


# clf_values = values[1700]


def convert_to_int(x):
    if x.dtype == str:
        return np.array([inv_terrains[t] for t in x])
    return x


ypred = convert_to_int(clf_values["pred"])
ytest = convert_to_int(clf_values["true"])
terr_pred = terrains[ypred]
terr_test = terrains[ytest]
terrains = [
    "CONCRETE",
    "DIRT_ROAD",
    "PLOUGHED",
    "UNPLOUGHED",
    "ASPHALT",
    "FLOORING",
    "ICE",
    "SANDY_LOAM",
    "SNOW",
]
vulpi_terrain = ["CONCRETE", "DIRT_ROAD", "PLOUGHED", "UNPLOUGHED"]
terr_test_bi = np.isin(terr_test, vulpi_terrain)
terr_pred_bi = np.isin(terr_pred, vulpi_terrain)

In [ ]:
clf_values["repr"].shape

In [ ]:
from sklearn.manifold import TSNE

k = 0

X = clf_values["repr"]
XX = clf_values["repr_train"]
XXX = clf_values["repr_val"]

ytest = convert_to_int(clf_values["true"])
yytest = convert_to_int(clf_values["true_train"])
yyytest = convert_to_int(clf_values["true_val"])

fold_size_x = X.shape[0] // 5
fold_size_xx = XX.shape[0] // 5
fold_size_xxx = XXX.shape[0] // 5

X = X[fold_size_x * k : fold_size_x * (k + 1)]
XX = XX[fold_size_xx * k : fold_size_xx * (k + 1)]
XXX = XXX[fold_size_xxx * k : fold_size_xxx * (k + 1)]
X = np.concatenate([X, XX, XXX], axis=0)
print(X.shape)

ytest = ytest[fold_size_x * k : fold_size_x * (k + 1)]
yytest = yytest[fold_size_xx * k : fold_size_xx * (k + 1)]
yyytest = yyytest[fold_size_xxx * k : fold_size_xxx * (k + 1)]

ytest = np.concatenate([ytest, yytest, yyytest], axis=0)
print(ytest.shape)
X_embedded = TSNE(n_components=2, perplexity=50, learning_rate=10).fit_transform(X)

# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# plt.show()

In [ ]:
from utils import constants

for i, terrain in enumerate(terrains):
    c = constants.PLOTCOLORS[terrain.lower().replace("_", " ")]
    # mark = "o" if terrain in vulpi_terrain
    plt.scatter(
        X_embedded[ytest == i, 0],
        X_embedded[ytest == i, 1],
        20,
        label=terrain,
        marker=".",
        c=c,
        rasterized=True,
    )
    if results_dir.stem == "vulpi":
        plt.xlim(-30, 40)
        plt.ylim(-30, 30)
    elif results_dir.stem == "husky":
        plt.xlim(-45, 45)
        plt.ylim(-45, 45)
    else:
        plt.xlim(-35, 35)
        plt.ylim(-35, 35)
    print(f"Generated for {terrain}")
    plt.plot()

plt.savefig(f"analysis/figs/figs-tsne/{results_dir.stem}_total.pdf")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.show()
# ytest
for i, terrain in enumerate(terrains):
    c = constants.PLOTCOLORS[terrain.lower().replace("_", " ")]
    plt.scatter(
        X_embedded[ytest == i, 0],
        X_embedded[ytest == i, 1],
        20,
        label=terrain,
        marker=".",
        c=c,
        rasterized=True,
    )
    if results_dir.stem == "vulpi":
        plt.xlim(-30, 40)
        plt.ylim(-30, 30)
    elif results_dir.stem == "husky":
        plt.xlim(-45, 45)
        plt.ylim(-45, 45)
    else:
        plt.xlim(-35, 35)
        plt.ylim(-35, 35)
    plt.savefig(f"analysis/figs/figs-tsne/{results_dir.stem}_{terrain}.pdf")
    print(f"Generated for {terrain}")
    plt.plot()
    plt.show()

In [ ]:
k = 0

X = clf_values["repr"]
XX = clf_values["repr_train"]
XXX = clf_values["repr_val"]

ytest = convert_to_int(clf_values["true"])
yytest = convert_to_int(clf_values["true_train"])
yyytest = convert_to_int(clf_values["true_val"])

fold_size_x = X.shape[0] // 5
fold_size_xx = XX.shape[0] // 5
fold_size_xxx = XXX.shape[0] // 5

X = X[fold_size_x * k : fold_size_x * (k + 1)]
XX = XX[fold_size_xx * k : fold_size_xx * (k + 1)]
XXX = XXX[fold_size_xxx * k : fold_size_xxx * (k + 1)]
X = np.concatenate([X, XX, XXX], axis=0)
print(X.shape)

ytest = ytest[fold_size_x * k : fold_size_x * (k + 1)]
yytest = yytest[fold_size_xx * k : fold_size_xx * (k + 1)]
yyytest = yyytest[fold_size_xxx * k : fold_size_xxx * (k + 1)]

ytest = np.concatenate([ytest, yytest, yyytest], axis=0)
print(ytest.shape)
X_embedded = TSNE(n_components=3, perplexity=50, learning_rate=10).fit_transform(X)

In [ ]:
import matplotlib.animation as animation

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(projection="3d")

pos = ax.get_position()
print(pos)
pos.x0 = 0.05
pos.x1 = 0.75
# pos.y0 = 0.05
# pos.y1 = 0.95
ax.set_position(pos)

for i, terrain in enumerate(terrains):
    terr = terrain.replace("_", " ").lower()
    col = constants.PLOTCOLORS[terr]
    lab = terr.replace("sandy", "silty").title()
    ax.scatter(
        X_embedded[ytest == i, 0],
        X_embedded[ytest == i, 1],
        X_embedded[ytest == i, 2],
        "o",
        label=lab,
        s=5,
        marker=".",
        c=col,
    )

ax.legend(loc="center left", bbox_to_anchor=(1.2, 0.5), markerscale=6)
# ax.set_xlabel("Component - 1")
# ax.set_ylabel("Component - 2")
# ax.set_zlabel("Component - 3")
# ax.show()


def rotate(angle):
    ax.view_init(azim=angle)


rot_animation = animation.FuncAnimation(
    fig,
    rotate,
    frames=np.arange(0, 362, 2),
    interval=100,
)
# rot_animation.save('rotation.gif', dpi=300, writer='imagemagick', extra_args=['-scale', '1920x1080'])

In [ ]:
rot_animation.save(
    "rotation_2.gif", dpi=300, writer="imagemagick", extra_args=["-scale", "960x540"]
)

In [ ]:
import matplotlib
import plotly.express as px

fig = px.scatter_3d(
    x=X_embedded[:, 0],
    y=X_embedded[:, 1],
    z=X_embedded[:, 2],
    color=[
        terrains[i].replace("_", " ").replace("sandy", "silty").title() for i in ytest
    ],
    color_discrete_sequence=[
        matplotlib.colors.to_hex(v) if not v.startswith("#") else v
        for v in constants.PLOTCOLORS.values()
    ],
)
fig.update_layout(
    scene=dict(
        xaxis=dict(title="Component 1"),
        yaxis=dict(title="Component 2"),
        zaxis=dict(title="Component 3"),
    )
)
fig.update_traces(marker=dict(size=2))
fig.show()

fig.write_html("../analysis/figs/tne_3d.html", include_plotlyjs="cdn")
fig.write_html(
    "../analysis/figs/tne_3d_embed.html", full_html=False, include_plotlyjs="cdn"
)

In [ ]:
from sklearn.manifold import TSNE

k = 0

X = clf_values["repr"]
num_pred = X.shape[0]
fold_size_x = num_pred // 5
X = X[fold_size_x * k : fold_size_x * (k + 1)]
yytest = ytest[fold_size_x * k : fold_size_x * (k + 1)]
X_embedded = TSNE(
    n_components=2, perplexity=50, learning_rate=10, n_iter=2000
).fit_transform(X)

# ytest
plt.plot(
    X_embedded[np.isin(yytest, vulpi_terrain), 0],
    X_embedded[np.isin(yytest, vulpi_terrain), 1],
    "o",
    label="VULPI",
)
plt.plot(
    X_embedded[~np.isin(yytest, vulpi_terrain), 0],
    X_embedded[~np.isin(yytest, vulpi_terrain), 1],
    "o",
    label="HUSKY",
)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.show()